### General comment section

In [1]:
# This notebook should be quite straightforward
# No particolar issues to highilight

#General comments:
# move function to utils.py module
# rewrite path for absolute path

In [1]:
import glob
import sys
import numpy as np
import imageio
import cv2
import random
from skimage import transform
import os
from tqdm import tqdm
from subprocess import check_output
import matplotlib.pyplot as plt
from importlib import import_module
import pickle
# from numba import jit
from skimage.morphology import erosion

In [2]:
# CHANGE PATH FOR GENERAL CODE
# CHANGE PATH FOR GENERAL CODE
# CHANGE PATH FOR GENERAL CODE
# CHANGE PATH FOR GENERAL CODE

LoadImagesForCrop =  'DATASET/TRAIN_VAL/all_images/images/'
LoadLabelsForCrop = 'DATASET/TRAIN_VAL/all_masks/masks/'
SaveCropImages =  'DATASET/TRAIN_VAL/all_cropped_images/images/' 
SaveCropMasks = 'DATASET/TRAIN_VAL/all_cropped_masks/masks/'

# CHANGE PATH FOR GENERAL CODE
# CHANGE PATH FOR GENERAL CODE
# CHANGE PATH FOR GENERAL CODE
# CHANGE PATH FOR GENERAL CODE

IMG_WIDTH = 1600
IMG_HEIGTH = 1200

# Crop size 
XCropSize = 512
YCropSize = 512
# if the coord are lesser than the crop size
# overlapping between crop is allowed
# if XCropCoord = XCropSize and same for Y coord, no overlapping beetween crop
XCropCoord = 400
YCropCoord = 400

XCropNum = int(IMG_WIDTH/XCropCoord)
YCropNum = int(IMG_HEIGTH/YCropCoord)

NumCropped = int(IMG_WIDTH/XCropCoord * IMG_HEIGTH/YCropCoord)

YShift = YCropSize - YCropCoord
XShift = XCropSize - XCropCoord

x_coord = [XCropCoord*i for i in range(0, XCropNum+1)]
y_coord = [YCropCoord*i for i in range(0, YCropNum+1)]

In [4]:
def cropper(image, mask):
    
    
    CroppedImgs = np.zeros((NumCropped, YCropSize, XCropSize, 3), np.uint8)
    CroppedMasks = np.zeros((NumCropped, YCropSize, XCropSize), np.uint8)
    idx = 0
    
    for i in range(0, 4):
        for j in range(0, 3):
                
                if (i == 0) & (j == 0):
                    CroppedImgs[idx] = image[y_coord[j]:y_coord[j+1] + YShift, x_coord[i]:x_coord[i+1] + XShift]
                    CroppedMasks[idx] = mask[y_coord[j]:y_coord[j+1] + YShift, x_coord[i]:x_coord[i+1] + XShift]
                    idx +=1 

                if (i == 0) & (j != 0):
                    CroppedImgs[idx] = image[y_coord[j] - YShift : y_coord[j+1], x_coord[i]:x_coord[i+1] + XShift]
                    CroppedMasks[idx] = mask[y_coord[j] - YShift : y_coord[j+1], x_coord[i]:x_coord[i+1] + XShift]
                    idx +=1 

                if (i != 0) &  (j == 0):
                    CroppedImgs[idx] = image[y_coord[j]:y_coord[j+1] + YShift, x_coord[i] - XShift :x_coord[i+1]]
                    CroppedMasks[idx] = mask[y_coord[j]:y_coord[j+1] + YShift, x_coord[i] - XShift :x_coord[i+1]]
                    idx +=1 

                if (i != 0) &  (j != 0):
                    CroppedImgs[idx] = image[y_coord[j] - YShift : y_coord[j+1], x_coord[i] - XShift :x_coord[i+1]]
                    CroppedMasks[idx] = mask[y_coord[j] - YShift : y_coord[j+1], x_coord[i] - XShift :x_coord[i+1]]
                    idx +=1   
            
    return CroppedImgs, CroppedMasks

In [5]:
image_ids = os.listdir(LoadImagesForCrop)

Number = [int(num.split('.')[0]) for num in image_ids]
Number.sort()

image_ids = [str(num) + '.tiff' for num in Number]

In [6]:
def read_image_labels(image_id):
     
        x = cv2.imread(LoadImagesForCrop + image_id)
        image = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(LoadLabelsForCrop + image_id)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        
        return image, mask

def make_cropper(image_ids, SaveCropImages, SaveCropMasks, shift = 0):
    ix = shift

    for ax_index, name in tqdm(enumerate(image_ids),total=len(image_ids)):
        
        if name == '252.tiff':
            print(ix)
        
        image, mask = read_image_labels(name)
        
        if int(name.split('.')[0]) <= 252:
            mask = erosion(np.squeeze(mask[:,:,0:1]), selem=np.ones([2,2]))
        else:
             mask = np.squeeze(mask[:,:,0:1])
            
       
        CroppedImages, CroppedMasks = cropper(image, mask)                                     
        
        for i in range(0,NumCropped):
            
            crop_imgs_dir = SaveCropImages + '{}.tiff'.format(ix)
            crop_masks_dir = SaveCropMasks + '{}.tiff'.format(ix) 

            plt.imsave(fname= crop_imgs_dir, arr = CroppedImages[i])
            plt.imsave(fname= crop_masks_dir,arr = CroppedMasks[i], cmap='gray')

            ix +=1
    return

In [7]:
make_cropper(image_ids, SaveCropImages,SaveCropMasks)

 91%|█████████ | 193/213 [02:09<00:11,  1.78it/s]

2316


100%|██████████| 213/213 [02:22<00:00,  1.50it/s]
